In [1]:
%%capture
!pip install mne
!pip install wandb

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# import mne
import pywt
import mne
import wandb


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'
RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
X = RAW['X'].astype('float64')
Y = RAW['Y'].astype('float64')
group = RAW['Group'].astype('float64')

In [5]:
X= np.moveaxis(X,1,2)
X.shape

(5834, 180, 29)

In [6]:
X_reshaped = X.reshape(X.shape[0],-1)
X_reshaped.shape

(5834, 5220)

In [7]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
# scaler = StandardScaler()

scaler= MinMaxScaler(feature_range=(0, 1))
scaled_data_reshaped = scaler.fit_transform(X_reshaped)

# Reshape back to original shape if necessary
scaled_data = scaled_data_reshaped.reshape(X.shape)

#Split the data

In [8]:
from sklearn.model_selection import KFold
import numpy as np

# Configuration
n_splits = 5  # Number of folds
random_state = 42
shuffle = True

# Initialize KFold

kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

# Create a placeholder array for fold assignments
fold_assignments = np.empty(len(Y), dtype=int)

# Split the data using kf.split. Note: kf.split does not need Y for splitting, but it's kept here for consistency
for fold, (train_idx, test_idx) in enumerate(kf.split(scaled_data, Y)):
    # Assign fold number
    fold_assignments[test_idx] = fold

# Example: To get the train and test sets for the first fold
fold_number = 0  # Choose which fold to use (0 to n_splits-1)
train_idx = np.where(fold_assignments != fold_number)[0]
test_idx = np.where(fold_assignments == fold_number)[0]

X_train, X_test = scaled_data[train_idx], scaled_data[test_idx]
Y_train, Y_test = Y[train_idx], Y[test_idx]

In [9]:

X_train.shape,Y_train.shape,X_test.shape

((4667, 180, 29), (4667,), (1167, 180, 29))

#CNN Model

In [10]:

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR API key must be 40 characters long, yours was 169
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


This is with using only the data with out any transformations applied. So This is just plain Deep learning.

In [13]:
from wandb.keras import WandbCallback
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import callbacks
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import  Conv1D,Conv1DTranspose,BatchNormalization,LeakyReLU,MaxPool1D,Flatten,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D,ReLU,MaxPooling1D,Conv2D,MaxPool2D,GlobalAveragePooling2D,\
AveragePooling2D,Bidirectional, LSTM, concatenate,Reshape
from tensorflow.keras.regularizers import l2
from keras.layers import ConvLSTM1D, LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from keras_radam import RAdam

checkpoint = ModelCheckpoint("/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/CNN1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.50,
                              patience = 10,
                              verbose = 1)

# tf_callback = tf.keras.callbacks.TensorBoard(
#                             log_dir="/content/drive/MyDrive/DeepLearning/TomatoLeaf/logs",
#                             histogram_freq=1,
#                             write_graph=True,
#                             write_images=True,
#                             update_freq="epoch"
#                             )

# we put our call backs into a callback list

# callbacks = [checkpoint, reduce_lr,WandbCallback()]
callbacks = []

In [20]:
wandb.init(project="Epileptic Seizure", entity="RaihanRabby", dir = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb')

accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇█▇█▇█████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▇▇▇█▇▇█▇████████████████████████
val_loss,████▇▇▆▅▃▃▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.93443
best_epoch,99
best_val_loss,0.37434
epoch,99
loss,0.3638
val_accuracy,0.92716


In [21]:
%%time
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import l2
def build_sequential_model(input_shape):

    clear_session()
    model = Sequential()



    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape),)
    model.add(BatchNormalization()),
    model.add(MaxPooling1D(2, padding="same")),
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same")),
    # model.add(BatchNormalization()),
    model.add(MaxPooling1D(2, padding="same")),
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same")),
    # model.add(BatchNormalization()),
    model.add(MaxPooling1D(2, padding="same"))


    model.add(Conv1D(filters=1024, kernel_size=3,strides =1, activation='relu'))

    model.add(LSTM(256, return_sequences=True, )),
    model.add(LSTM(256, return_sequences=False))


    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    # model.add(MaxPool1D(pool_size=4,padding = 'same'))

    # model.add(Dropout(0.5))

    # Flatten for Dense layers after Conv1D/LSTM processing

    model.add(Flatten())
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid',kernel_regularizer=l2(0.001)))

    opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer = opt,
        loss='binary_crossentropy',
        metrics=['accuracy'])
    model.summary()
    return model
latent_dim = 32
Model = build_sequential_model(input_shape=(180,29))
Model.fit(X_train, Y_train, epochs=100, batch_size=128, callbacks=[WandbCallback()], validation_data = (X_test, Y_test), shuffle=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 180, 256)          22528     
                                                                 
 batch_normalization (Batch  (None, 180, 256)          1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 90, 256)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 90, 512)           393728    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 45, 512)           0         
 g1D)                                                            
                                                        

36/37 [============================>.] - ETA: 0s - loss: 1.0032 - accuracy: 0.5970

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240520_123744-xyfxzsn6/files/model-best)... Done. 0.3s


37/37 [==============================] - 19s 290ms/step - loss: 1.0031 - accuracy: 0.5959 - val_loss: 1.0053 - val_accuracy: 0.5030
Epoch 2/100
36/37 [============================>.] - ETA: 0s - loss: 0.9928 - accuracy: 0.6775

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240520_123744-xyfxzsn6/files/model-best)... Done. 0.3s


...

37/37 [==============================] - 11s 315ms/step - loss: 0.4201 - accuracy: 0.9291 - val_loss: 0.4225 - val_accuracy: 0.9254
Epoch 89/100
37/37 [==============================] - 3s 67ms/step - loss: 0.4121 - accuracy: 0.9331 - val_loss: 0.4236 - val_accuracy: 0.9229
Epoch 90/100
37/37 [==============================] - 2s 60ms/step - loss: 0.4090 - accuracy: 0.9329 - val_loss: 0.4276 - val_accuracy: 0.9237
Epoch 91/100
37/37 [==============================] - 2s 58ms/step - loss: 0.4041 - accuracy: 0.9379 - val_loss: 0.4268 - val_accuracy: 0.9195
Epoch 92/100
37/37 [==============================] - 2s 58ms/step - loss: 0.4412 - accuracy: 0.9160 - val_loss: 0.5074 - val_accuracy: 0.8835
Epoch 93/100
36/37 [============================>.] - ETA: 0s - loss: 0.4412 - accuracy: 0.9158

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240520_123744-xyfxzsn6/files/model-best)... Done. 0.3s


37/37 [==============================] - 12s 321ms/step - loss: 0.4404 - accuracy: 0.9162 - val_loss: 0.4183 - val_accuracy: 0.9246
Epoch 94/100
37/37 [==============================] - 2s 63ms/step - loss: 0.4083 - accuracy: 0.9325 - val_loss: 0.4206 - val_accuracy: 0.9254
Epoch 95/100
37/37 [==============================] - 2s 57ms/step - loss: 0.4033 - accuracy: 0.9357 - val_loss: 0.4223 - val_accuracy: 0.9229
Epoch 96/100
37/37 [==============================] - 2s 59ms/step - loss: 0.4057 - accuracy: 0.9334 - val_loss: 0.4327 - val_accuracy: 0.9195
Epoch 97/100
37/37 [==============================] - 2s 66ms/step - loss: 0.4250 - accuracy: 0.9226 - val_loss: 0.4185 - val_accuracy: 0.9246
Epoch 98/100
37/37 [==============================] - 2s 64ms/step - loss: 0.4023 - accuracy: 0.9340 - val_loss: 0.4434 - val_accuracy: 0.9117
Epoch 99/100
37/37 [==============================] - 2s 62ms/step - loss: 0.4037 - accuracy: 0.9321 - val_loss: 0.4657 - val_accuracy: 0.9040
Epoch 100/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb/wandb/run-20240520_123744-xyfxzsn6/files/model-best)... Done. 0.4s


37/37 [==============================] - 14s 379ms/step - loss: 0.4142 - accuracy: 0.9250 - val_loss: 0.4137 - val_accuracy: 0.9237
CPU times: user 8min 51s, sys: 20.2 s, total: 9min 11s
Wall time: 11min 28s


In [ ]:

pro_pred = Model.predict(X_test)

37/37 [==============================] - 0s 6ms/step


In [ ]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             confusion_matrix, cohen_kappa_score, matthews_corrcoef)


# Convert probabilities to binary predictions with a threshold of 0.5
y_pred = (pro_pred >= 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
roc_auc = roc_auc_score(Y_test, pro_pred)
cm = confusion_matrix(Y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])  # TN / (TN + FP)
sensitivity = recall  # Sensitivity is the same as recall
kappa = cohen_kappa_score(Y_test, y_pred)
mcc = matthews_corrcoef(Y_test, y_pred)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'accuracy': [accuracy],
    'precision': [precision],
    'recall': [recall],
    'f1': [f1],
    'roc_auc': [roc_auc],
    'specificity': [specificity],
    'sensitivity': [sensitivity],
    'kappa': [kappa],
    'mcc': [mcc]
})

# Print the DataFrame
print(metrics_df)

   accuracy  precision   recall        f1   roc_auc  specificity  sensitivity  \
0  0.924593   0.966355  0.88075  0.921569  0.965541     0.968966      0.88075   

      kappa      mcc  
0  0.849261  0.85265  


In [ ]:
metrics_df

,accuracy,precision,recall,f1,roc_auc,specificity,sensitivity,kappa,mcc
0,0.924593,0.966355,0.88075,0.921569,0.965541,0.968966,0.88075,0.849261,0.85265
